# Extracting minimizers

**Adapted from**: Example 6.23 of [L09]

[L09] Laurent, Monique.
*Sums of squares, moment matrices and optimization over polynomials.*
Emerging applications of algebraic geometry (2009): 157-270.

## Introduction

Consider the polynomial optimization problem [L09, Example 6.23] of
minimizing the linear function $-x_1 - x_2$
over the basic semialgebraic set defined by the inequalities
$x_2 \le 2x_1^4 - 8x_1^3 + 8x_1^2 + 2$,
$x_2 \le 4x_1^4 - 32x_1^3 + 88x_1^2 - 96x_1 + 36$ and the box constraints
$0 \le x_1 \le 3$ and $0 \le x_2 \le 4$,
World Scientific, **2009**.

In [1]:
using DynamicPolynomials
@polyvar x[1:2]
p = -sum(x)
using SumOfSquares
K = @set x[1] >= 0 && x[1] <= 3 && x[2] >= 0 && x[2] <= 4 && x[2] <= 2x[1]^4 - 8x[1]^3 + 8x[1]^2 + 2 && x[2] <= 4x[1]^4 - 32x[1]^3 + 88x[1]^2 - 96x[1] + 36

Basic semialgebraic Set defined by no equality
6 inequalities
 x[1] ≥ 0
 -x[1] + 3 ≥ 0
 x[2] ≥ 0
 -x[2] + 4 ≥ 0
 2*x[1]^4 - 8*x[1]^3 + 8*x[1]^2 - x[2] + 2 ≥ 0
 4*x[1]^4 - 32*x[1]^3 + 88*x[1]^2 - 96*x[1] - x[2] + 36 ≥ 0


We will now see how to find the optimal solution using Sum of Squares Programming.
We first need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v0.21.6/installation/#Supported-solvers) for a list of the available choices.

In [2]:
import CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)

MathOptInterface.OptimizerWithAttributes(CSDP.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.Silent() => true])

A Sum-of-Squares certificate that $p \ge \alpha$ over the domain `S`, ensures that $\alpha$ is a lower bound to the polynomial optimization problem.
The following function searches for the largest lower bound and finds zero using the `d`th level of the hierarchy`.

In [3]:
function solve(d)
    model = SOSModel(solver)
    @variable(model, α)
    @objective(model, Max, α)
    @constraint(model, c, p >= α, domain = K, maxdegree = d)
    optimize!(model)
    println(solution_summary(model))
    return model
end

solve (generic function with 1 method)

The first level of the hierarchy gives a lower bound of `-7``

In [4]:
model4 = solve(4)

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -7.00000e+00
  Dual objective value : -7.00000e+00

* Work counters
  Solve time (sec)   : 1.15430e-02



A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: VariableRef
`Vector{AffExpr}`-in-`SumOfSquares.SOSPolynomialSet{BasicSemialgebraicSet{Int64, DynamicPolynomials.Polynomial{true, Int64}, FullSpace}, DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}, SumOfSquares.Certificate.Putinar{SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, Tuple{}}, SOSCone, MonomialBasis}}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CSDP
Names registered in the model: c, α

The second level improves the lower bound but still does not provide the solution

In [5]:
model5 = solve(5)
ν5 = moment_matrix(model5[:c])
extractatoms(ν5, 1e-2)
SumOfSquares.MultivariateMoments.computesupport!(ν5, 1e-1)
ν5.support

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -6.66667e+00
  Dual objective value : -6.66667e+00

* Work counters
  Solve time (sec)   : 3.86181e-02



Algebraic Set defined by 5 equalities
 -x[2] + 3.9999999953252514 = 0
 -x[1] + 2.805349350891122 = 0
 -x[2]^2 + 15.999999969606513 = 0
 -x[1]*x[2] + 11.221397393608639 = 0
 -x[1]^2 + 8.416048039737186 = 0


The third level finds the optimal objective value as lower bound...

In [6]:
model7 = solve(7)

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : -5.50801e+00
  Dual objective value : -5.50801e+00

* Work counters
  Solve time (sec)   : 2.55663e-01



A JuMP Model
Maximization problem with:
Variable: 1
Objective function type: VariableRef
`Vector{AffExpr}`-in-`SumOfSquares.SOSPolynomialSet{BasicSemialgebraicSet{Int64, DynamicPolynomials.Polynomial{true, Int64}, FullSpace}, DynamicPolynomials.Monomial{true}, DynamicPolynomials.MonomialVector{true}, SumOfSquares.Certificate.Putinar{SumOfSquares.Certificate.Newton{SOSCone, MonomialBasis, Tuple{}}, SOSCone, MonomialBasis}}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: CSDP
Names registered in the model: c, α

...and proves it by exhibiting the minimizer.

In [7]:
ν7 = moment_matrix(model7[:c])
η = extractatoms(ν7, 1e-3) # Returns nothing as the dual is not atomic

Atomic measure on the variables x[1], x[2] with 1 atoms:
 at [2.329520196265274, 3.1784930773209235] with weight 0.9999999991472346

We can indeed verify that the objective value at `x_opt` is equal to the lower bound.

In [8]:
x_opt = η.atoms[1].center
p(x_opt)

-5.508013273586197

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*